## 라이브러리 및 데이터 로드

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import joblib

In [2]:
# 스케일링 데이터
st_df_test = pd.read_csv(r'st_df_test.csv', index_col = 0)

# 원본 데이터
st_df_test_orginal = pd.read_csv(r'original_df_test.csv', index_col = 0)
st_df_test_orginal=st_df_test_orginal.loc[st_df_test_orginal.index.isin(st_df_test.index)]

## 클러스터 모델 GMM

In [5]:
# 학습 모델 불러오기
GMM_from_joblib = joblib.load('GMM_model.pkl') 
result_GMM=GMM_from_joblib.predict(st_df_test.iloc[:,:31])

#result_GMM=GMM_model.predict(st_df_test.iloc[:,:31])

## 클러스터 모델 CNN

In [11]:
# 학습모델 불러오기

from keras.models import load_weight
CNN_model = load_weights('CNN_model.h5')

ImportError: cannot import name 'load_weight' from 'keras.models' (C:\acaconda\lib\site-packages\keras\models.py)

In [ ]:

test = st_df_test.iloc[:,:31]
test=np.array(test).reshape(len(test),1,31,1)
predicted_clst=CNN_model.predict(test)

predicted_clst_df=pd.DataFrame(predicted_clst)
result_CNN = predicted_clst_df.idxmax(axis=1)

## 가중치 모델 35분 예측 

In [ ]:
# distance 적용 클러스터별 35분 예측값
import math
def predict_w_distance(data, predict_data, num):
    df = data[data['clst'] == num].reset_index(drop = True)

    columns = df.columns
    data_35 = df[['09:35']]
    
    df = df[columns[:31]]
    data_size = len(df)

    distances = []

    for row in np.array(df):
        dis = np.linalg.norm(row-predict_data)
        distances.append(dis)

    weights = 1/np.log(np.array(distances)+1)
    weights_sum = np.sum(weights)
    weights = weights/weights_sum

    result = 0

    for i in range(len(distances)):
        result += weights[i]*data_35.iloc[i][0]
    
    return result

In [ ]:
predict_35 = []
columns = st_df_test.columns

test_set_30 = np.array(st_df_test[columns[:31]])

for i in range(len(st_df_test)):
    cluster_num = result_GMM[i]
    
    predict_35.append(predict_w_distance(train_set,test_set_30[i],cluster_num))

## 평가

In [ ]:
clst_price = pd.DataFrame()
clst_price['clst_num'] = result_GMM
clst_price['predict_35'] = predict_35
clst_price['real_value'] = st_df_test['09:35'].values.tolist()
clst_price['diff'] = clst_price['real_value'] - clst_price['predict_35']
clst_price=clst_price[(result_GMM==result_CNN)]

print('MAE : ',np.mean(abs(clst_price['diff'])))
print('GMM과 CNN에서 겹치는 행의 수 : ',sum(result_GMM==result_CNN))

In [ ]:
st_df_test_orginal=st_df_test_orginal[(result_CNN==result_GMM).values]

In [ ]:
def get_revenue(predict_35, test_set_original):
    revenues = []
    original_30 = test_set_original['09:30']
    original_35 = test_set_original['09:35']
    
    k=0
    for i in range(len(predict_35)):
        if predict_35[i] > 1.0:
            rev = (original_35 [i]- original_30[i])/original_30[i]
            revenues.append(rev*100)
            k+=1
                        
    
    print('추천 종목 수 : ',k)
    return revenues

rev_percent= get_revenue(clst_price['predict_35'].tolist(),st_df_test_orginal)
print('예상 수익률 : ',round(sum(rev_percent),4))

In [ ]:
st_df_test['pred_35']=predict_35
test_set_2=st_df_test[(result_CNN==result_GMM).values]
predicted_stock = test_set_2.loc[st_df_test['pred_35']>1.0]
print('추천 종목 : ',predicted_stock.index)